In [1]:
# %% [markdown]
# # Jupyter Notebook Loading Header
#
# This is a custom loading header for Jupyter Notebooks in Visual Studio Code.
# It includes common imports and settings to get you started quickly.

# %% [markdown]
## Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.cloud import bigquery
import os
path = r'C:\Users\DwaipayanChakroborti\AppData\Roaming\gcloud\legacy_credentials\dchakroborti@tonikbank.com\adc.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = path
client = bigquery.Client(project='prj-prod-dataplatform')

# %% [markdown]
## Configure Settings
# Set options or configurations as needed
# Example: pd.set_option('display.max_columns', None)

# Customers who reported to be Defrauded or Hacked - get this list from the Ops team

In [2]:
dfhacked = pd.read_excel(r"C:\Users\DwaipayanChakroborti\OneDrive - Tonik Financial Pte Ltd\MyStuff\Biswa\Demo&Telco Scorecard\Data\SupportingFile\Database of internal & external fraud incidents v2.xlsx", sheet_name = 'Main')

c:\Users\DwaipayanChakroborti\Myenv\reporting\lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [3]:
dfhacked.columns

Index(['Source', 'Ticket Number', 'Ticket Status', 'Report Date', 'Fraud Type',
       'Sub MO', 'Disputed Amount', 'OFI', 'RFI', 'Complainant Name',
       'Contact Information', 'Source Account Number', 'With Police Report?',
       'TSA/Account Number', 'PID', 'Account Name', 'Segment/KYC', 'Address1',
       'Address2', 'Address3', 'Mobile Number', 'Account Status',
       'Date Blocked', 'TDB Account Balance', 'With TD?', 'With Active Loans?',
       'Callout Result', 'Disposition', 'Actions'],
      dtype='object')

In [4]:
pd.set_option("Display.max_columns", None)
dfhacked.head()

,Source,Ticket Number,Ticket Status,Report Date,Fraud Type,Sub MO,Disputed Amount,OFI,RFI,Complainant Name,Contact Information,Source Account Number,With Police Report?,TSA/Account Number,PID,Account Name,Segment/KYC,Address1,Address2,Address3,Mobile Number,Account Status,Date Blocked,TDB Account Balance,With TD?,With Active Loans?,Callout Result,Disposition,Actions
0,JIRA,CC-1686423,Closed,2023-05-22,Internal Fraud,Advance Fee Scam,1500.0,Various,CIMB via DragonPay,KATHLEEN ROSE BAUTISTA ANDAL,NaN,Various,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,Valid,"Joint investigation of InfoSec, AF Ops and HR ..."
1,NaN,CC-1698906,Closed,2023-05-26,NaN,NaN,1500.0,NaN,NaN,JESSA NA NAZ,NaN,NaN,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,CC-1698935,Closed,2023-05-26,NaN,NaN,2000.0,NaN,NaN,ROLLEN CARILLO REGINO,NaN,NaN,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,CC-1698352,Closed,2023-05-26,NaN,NaN,1500.0,NaN,NaN,JAIME FELIX REY III GENERAO CRUZ,NaN,NaN,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,CC-1698914,Closed,2023-05-26,NaN,NaN,1500.0,NaN,NaN,MAYDEN CORPUZ MAGLACION,NaN,NaN,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
dfhacked['Disposition'].value_counts()

Disposition
Not Valid    162
Valid          2
Name: count, dtype: int64

In [6]:
dfhacked = dfhacked[dfhacked['Disposition']=='Valid']
dfhacked.shape

(2, 29)

In [7]:
# Replace spaces with underscores in column names
dfhacked.columns = dfhacked.columns.str.replace(' ', '_')
dfhacked.columns = dfhacked.columns.str.replace('/', '_')
dfhacked.columns = dfhacked.columns.str.replace('?', '')

In [8]:
dfhacked.dtypes

Source                           object
Ticket_Number                    object
Ticket_Status                    object
Report_Date              datetime64[ns]
Fraud_Type                       object
Sub_MO                           object
Disputed_Amount                 float64
OFI                              object
RFI                              object
Complainant_Name                 object
Contact_Information              object
Source_Account_Number            object
With_Police_Report               object
TSA_Account_Number              float64
PID                             float64
Account_Name                     object
Segment_KYC                      object
Address1                         object
Address2                         object
Address3                         object
Mobile_Number                   float64
Account_Status                   object
Date_Blocked             datetime64[ns]
TDB_Account_Balance              object
With_TD                          object


In [9]:
dfhacked1 = dfhacked[['Source', 'Ticket_Number', 'Ticket_Status', 'Report_Date', 'Fraud_Type',
       'Sub_MO', 'Disputed_Amount', 'OFI', 'RFI', 'Complainant_Name',
       'Contact_Information', 'With_Police_Report',
       'TSA_Account_Number', 'PID', 'Account_Name', 'Segment_KYC', 'Address1',
       'Address2', 'Address3', 'Mobile_Number', 'Account_Status',
       'Date_Blocked', 'With_TD', 'With_Active_Loans',
       'Callout_Result', 'Disposition', 'Actions']].copy()

In [10]:
from google.cloud import bigquery

# Define the dataset and table references
dataset_id = 'dap_ds_poweruser_playground'
table_id = 'Defraudedandhacked_mu'

# Define the table schema as per your DataFrame columns
schema = [
    bigquery.SchemaField("Source", "STRING"),
    bigquery.SchemaField("Ticket_Number", "STRING"),
    bigquery.SchemaField("Ticket_Status", "STRING"),
    bigquery.SchemaField("Report_Date", "TIMESTAMP"),
    bigquery.SchemaField("Fraud_Type", "STRING"),
    bigquery.SchemaField("Sub_MO", "STRING"),
    bigquery.SchemaField("Disputed_Amount", "FLOAT"),
    bigquery.SchemaField("OFI", "STRING"),
    bigquery.SchemaField("RFI", "STRING"),
    bigquery.SchemaField("Complainant_Name", "STRING"),
    bigquery.SchemaField("Contact_Information", "STRING"),
    bigquery.SchemaField("With_Police_Report", "STRING"),
    bigquery.SchemaField("TSA_Account_Number", "FLOAT"),
    bigquery.SchemaField("PID", "FLOAT"),
    bigquery.SchemaField("Account_Name", "STRING"),
    bigquery.SchemaField("Segment_KYC", "STRING"),
    bigquery.SchemaField("Address1", "STRING"),
    bigquery.SchemaField("Address2", "STRING"),
    bigquery.SchemaField("Address3", "STRING"),
    bigquery.SchemaField("Mobile_Number", "FLOAT"),
    bigquery.SchemaField("Account_Status", "STRING"),
    bigquery.SchemaField("Date_Blocked", "TIMESTAMP"),
    bigquery.SchemaField("With_TD", "STRING"),
    bigquery.SchemaField("With_Active_Loans", "STRING"),
    bigquery.SchemaField("Callout_Result", "STRING"),
    bigquery.SchemaField("Disposition", "STRING"),
    bigquery.SchemaField("Actions", "STRING"),
]

# Create the dataset reference
dataset_ref = client.dataset(dataset_id)

# Define the table reference
table_ref = dataset_ref.table(table_id)

# Configure the job to overwrite the table if it already exists
job_config = bigquery.LoadJobConfig(schema=schema)
job_config.autodetect = False
job_config.write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE

# Convert datetime columns to timestamp
for col in dfhacked.select_dtypes(include=['datetime64']):
    dfhacked[col] = dfhacked[col].astype('int64') // 10**9

# Load the DataFrame into BigQuery
job = client.load_table_from_dataframe(dfhacked1, table_ref, job_config=job_config)

# Wait for the job to complete
job.result()

print(f"Table {table_id} created in dataset {dataset_id}.")

Table Defraudedandhacked_mu created in dataset dap_ds_poweruser_playground.


In [11]:
sq = """select * from dap_ds_poweruser_playground.Defraudedandhacked_mu;"""
checkdf = client.query(sq).to_dataframe()
checkdf.shape

(2, 27)

# Query for Aquisition Channel

In [12]:
sq = """WITH installs AS 
(
SELECT
Event_Time,
CASE 
    ### social media ###
    -- anything from social
    WHEN LOWER(COALESCE(Media_Source,Partner,Campaign)) LIKE '%social%' THEN 'Social'
    ### direct ###
    -- fwb / referral
    WHEN COALESCE(Media_Source,Partner,Campaign) = 'af_app_invites' THEN 'FWB'
    WHEN (Media_Source = 'invalid_media_source_name' OR Media_Source IS NULL) AND LOWER(Campaign) LIKE '%refer%' THEN 'FWB'
    WHEN LOWER(COALESCE(Media_Source,Partner,Campaign)) LIKE '%refer%' THEN 'FWB'
    WHEN LOWER(COALESCE(Media_Source,Partner,Campaign)) LIKE '%moengage%' THEN 'FWB'
    -- apple search
    WHEN LOWER(COALESCE(Media_Source,Partner,Campaign)) LIKE '%apple search ads%' THEN 'AppleSearch'
    -- facebook / meta
    WHEN COALESCE(Media_Source,Partner,Campaign) LIKE '%Facebook Ads%' THEN 'Ads_Meta'
    WHEN COALESCE(Media_Source,Partner,Campaign) LIKE '%meta%' THEN 'Ads_Meta'
    WHEN COALESCE(Media_Source,Partner,Campaign) = 'facebook' THEN 'Ads_Meta'
    WHEN LOWER(COALESCE(Media_Source,Partner,Campaign)) = 'restricted' THEN 'Ads_Meta'
    -- google
    WHEN COALESCE(Media_Source,Partner,Campaign) LIKE '%googleadwords%' THEN 'Ads_Google'
      -- tiktok
    WHEN COALESCE(Media_Source,Partner,Campaign) like '%bytedanceglobal%' THEN 'TikTok_Paid'
    ### affiliates ###
    -- pokkt source
    WHEN LOWER(COALESCE(Media_Source,Partner,Campaign)) LIKE '%pokkt%' THEN 'Aff_Pokkt'
    WHEN COALESCE(Media_Source,Partner,Campaign) LIKE '%Tonik_CPA%' THEN 'Aff_Pokkt'
    WHEN COALESCE(Media_Source,Partner,Campaign) LIKE '%campaign_name%' THEN 'Aff_Pokkt'
    -- tyr ads
    WHEN LOWER(COALESCE(Media_Source,Partner,Campaign)) IN ('tyrads_int','tjzymob_int','tyrads','ta_tonik_aos_ph') THEN 'Aff_TyrAds'
    -- sales doubler
    WHEN LOWER(COALESCE(Media_Source,Partner,Campaign)) LIKE '%salesdoubler%' THEN 'Aff_SalesDoubler'
     
    -- imoney
    WHEN COALESCE(Media_Source,Partner,Campaign) LIKE '%imoney%' THEN 'Aff_iMoney'
    -- moneymax
    WHEN COALESCE(Media_Source,Partner,Campaign) LIKE '%moneymax%' THEN 'Aff_MoneyMax'
    -- jeff
    WHEN COALESCE(Media_Source,Partner,Campaign) LIKE '%jeff%' THEN 'Aff_Jeff'
    -- shareit
    WHEN COALESCE(Media_Source,Partner,Campaign) LIKE '%shareit%' THEN 'Aff_ShareIt'
     
    -- appnext
    WHEN COALESCE(Media_Source,Partner,Campaign) LIKE '%appnext%' THEN 'Aff_AppNext'
    -- mediadonuts / entravision
    WHEN LOWER(COALESCE(Media_Source,Partner,Campaign)) LIKE '%mediadonuts%' THEN 'Aff_Mediadonuts'
    WHEN LOWER(COALESCE(Media_Source,Partner,Campaign)) LIKE '%entravision%' THEN 'Aff_Mediadonuts'
    -- shopback
    WHEN LOWER(COALESCE(Media_Source,Partner,Campaign)) LIKE '%shopback_int%' THEN 'Aff_Shopback'
    -- deepsea
    WHEN LOWER(COALESCE(Media_Source,Partner,Campaign)) LIKE '%deepsea%' THEN 'Aff_DeepSea'
    ## organic ##
    WHEN COALESCE(Media_Source,Partner,Campaign) IN ('af_banner') THEN 'af_Banner'
    WHEN COALESCE(Media_Source,Partner,Campaign) LIKE '%sendgrid%' THEN 'Organic'
    WHEN COALESCE(Media_Source,Partner,Campaign) IN ('Website') THEN 'Website'
    
    ## owned media ##
    WHEN media_source IN ('PRFlexLoans_WebAndroidPressRelease') THEN 'Owned Media'
    
    WHEN media_source IN ('invalid_media_source_name') THEN 'Invalid Media Source'
    WHEN media_source IN ('None') THEN 'Broken OneLink'
ELSE "Organic"
END as Source,
AppsFlyer_ID,
Customer_User_ID,
Device_Type
    
FROM
(
    SELECT Install_Time,Event_Time,AppsFlyer_ID,Customer_User_ID,Platform,Partner,Media_Source,Campaign,Device_Type,OS_Version
    FROM `prj-prod-dataplatform.appsflyer_raw.organic_installs_report` I --Organic Installs
    WHERE Date(Event_Time) BETWEEN DATE_SUB(CURRENT_DATE, INTERVAL 11 MONTH) AND DATE_SUB(CURRENT_DATE, INTERVAL 1 DAY)
    AND DATE(_PARTITIONTIME) BETWEEN DATE_SUB(CURRENT_DATE, INTERVAL 350 DAY) AND DATE_SUB(CURRENT_DATE, INTERVAL 1 DAY)
    UNION ALL
    SELECT Install_Time,Event_Time,AppsFlyer_ID,Customer_User_ID,Platform,Partner,Media_Source,Campaign,Device_Type,OS_Version
    FROM `prj-prod-dataplatform.appsflyer_raw.installs_report` I -- Non-Organic Installs
    WHERE Date(Event_Time) BETWEEN DATE_SUB(CURRENT_DATE, INTERVAL 11 MONTH) AND DATE_SUB(CURRENT_DATE, INTERVAL 1 DAY)
    AND DATE(_PARTITIONTIME) BETWEEN DATE_SUB(CURRENT_DATE, INTERVAL 350 DAY) AND DATE_SUB(CURRENT_DATE, INTERVAL 1 DAY)
    
)
WHERE 1=1
AND event_time >= '2022-09-01'
)
, registrations AS 
(
SELECT
MAX(Event_Time) event_time,
CASE 
    ### social media ###
    -- anything from social
    WHEN LOWER(COALESCE(Media_Source,Partner,Campaign)) LIKE '%social%' THEN 'Social'
    ### direct ###
    -- fwb / referral
    WHEN COALESCE(Media_Source,Partner,Campaign) = 'af_app_invites' THEN 'FWB'
    WHEN (Media_Source = 'invalid_media_source_name' OR Media_Source IS NULL) AND LOWER(Campaign) LIKE '%refer%' THEN 'FWB'
    WHEN LOWER(COALESCE(Media_Source,Partner,Campaign)) LIKE '%refer%' THEN 'FWB'
    WHEN LOWER(COALESCE(Media_Source,Partner,Campaign)) LIKE '%moengage%' THEN 'FWB'
    -- apple search
    WHEN LOWER(COALESCE(Media_Source,Partner,Campaign)) LIKE '%apple search ads%' THEN 'AppleSearch'
    -- facebook / meta
    WHEN COALESCE(Media_Source,Partner,Campaign) LIKE '%Facebook Ads%' THEN 'Ads_Meta'
    WHEN COALESCE(Media_Source,Partner,Campaign) LIKE '%meta%' THEN 'Ads_Meta'
    WHEN COALESCE(Media_Source,Partner,Campaign) = 'facebook' THEN 'Ads_Meta'
    WHEN LOWER(COALESCE(Media_Source,Partner,Campaign)) = 'restricted' THEN 'Ads_Meta'
    -- google
    WHEN COALESCE(Media_Source,Partner,Campaign) LIKE '%googleadwords%' THEN 'Ads_Google'
      -- tiktok
    WHEN COALESCE(Media_Source,Partner,Campaign) like '%bytedanceglobal%' THEN 'TikTok_Paid'
    ### affiliates ###
    -- pokkt source
    WHEN LOWER(COALESCE(Media_Source,Partner,Campaign)) LIKE '%pokkt%' THEN 'Aff_Pokkt'
    WHEN COALESCE(Media_Source,Partner,Campaign) LIKE '%Tonik_CPA%' THEN 'Aff_Pokkt'
    WHEN COALESCE(Media_Source,Partner,Campaign) LIKE '%campaign_name%' THEN 'Aff_Pokkt'
    -- tyr ads
    WHEN LOWER(COALESCE(Media_Source,Partner,Campaign)) IN ('tyrads_int','tjzymob_int','tyrads','ta_tonik_aos_ph') THEN 'Aff_TyrAds'
    -- sales doubler
    WHEN LOWER(COALESCE(Media_Source,Partner,Campaign)) LIKE '%salesdoubler%' THEN 'Aff_SalesDoubler'
     
    -- imoney
    WHEN COALESCE(Media_Source,Partner,Campaign) LIKE '%imoney%' THEN 'Aff_iMoney'
    -- moneymax
    WHEN COALESCE(Media_Source,Partner,Campaign) LIKE '%moneymax%' THEN 'Aff_MoneyMax'
    -- jeff
    WHEN COALESCE(Media_Source,Partner,Campaign) LIKE '%jeff%' THEN 'Aff_Jeff'
    -- shareit
    WHEN COALESCE(Media_Source,Partner,Campaign) LIKE '%shareit%' THEN 'Aff_ShareIt'
     
    -- appnext
    WHEN COALESCE(Media_Source,Partner,Campaign) LIKE '%appnext%' THEN 'Aff_AppNext'
    -- mediadonuts / entravision
    WHEN LOWER(COALESCE(Media_Source,Partner,Campaign)) LIKE '%mediadonuts%' THEN 'Aff_Mediadonuts'
    WHEN LOWER(COALESCE(Media_Source,Partner,Campaign)) LIKE '%entravision%' THEN 'Aff_Mediadonuts'
    -- shopback
    WHEN LOWER(COALESCE(Media_Source,Partner,Campaign)) LIKE '%shopback_int%' THEN 'Aff_Shopback'
    -- deepsea
    WHEN LOWER(COALESCE(Media_Source,Partner,Campaign)) LIKE '%deepsea%' THEN 'Aff_DeepSea'
    ## organic ##
    WHEN COALESCE(Media_Source,Partner,Campaign) IN ('af_banner') THEN 'af_Banner'
    WHEN COALESCE(Media_Source,Partner,Campaign) LIKE '%sendgrid%' THEN 'Organic'
    WHEN COALESCE(Media_Source,Partner,Campaign) IN ('Website') THEN 'Website'
    
    ## owned media ##
    WHEN media_source IN ('PRFlexLoans_WebAndroidPressRelease') THEN 'Owned Media'
    
    WHEN media_source IN ('invalid_media_source_name') THEN 'Invalid Media Source'
    WHEN media_source IN ('None') THEN 'Broken OneLink'
ELSE "Organic"
END as Source,
AppsFlyer_ID,
Device_Type,
-- Customer_User_ID
FROM
(

  SELECT MIN(Event_time) event_time,Media_source,Campaign,Partner,AppsFlyer_ID,Install_Time,Platform,OS_Version,Device_Type
  FROM `prj-prod-dataplatform.appsflyer_raw.in_app_events_report`  -- Non-Organic Registrations
  WHERE (Event_Name='Onboarding_Success' OR Event_Name LIKE 'Onboarding_Success_Dashboard%')
  AND DATE(_PARTITIONTIME) BETWEEN DATE_SUB(CURRENT_DATE, INTERVAL 350 DAY) AND DATE_SUB(CURRENT_DATE, INTERVAL 1 DAY)
  AND DATE(Event_Time) BETWEEN DATE_SUB(CURRENT_DATE, INTERVAL 11 MONTH) AND DATE_SUB(CURRENT_DATE, INTERVAL 1 DAY)
  GROUP BY 2,3,4,5,6,7,8,9
  UNION ALL
      
  SELECT MIN(Event_time) event_time,Media_source,Campaign,Partner,AppsFlyer_ID,Install_Time,Platform,OS_Version,Device_Type
  FROM `prj-prod-dataplatform.appsflyer_raw.organic_in_app_events_report`  -- Organic Registrations
  WHERE (Event_Name='Onboarding_Success' OR Event_Name LIKE 'Onboarding_Success_Dashboard%')
  AND DATE(_PARTITIONTIME) BETWEEN DATE_SUB(CURRENT_DATE, INTERVAL 380 DAY) AND DATE_SUB(CURRENT_DATE, INTERVAL 1 DAY)
  AND DATE(Event_Time) BETWEEN DATE_SUB(CURRENT_DATE, INTERVAL 11 MONTH) AND DATE_SUB(CURRENT_DATE, INTERVAL 1 DAY)
  GROUP BY 2,3,4,5,6,7,8,9
    
)
WHERE 1=1
AND event_time >= '2022-09-01'
GROUP BY 2,3,4
)
, applications AS 
(
SELECT 
decision_date,
IF(reloan_flag=1,'Re-loan',new_loan_type) loan_type,
CASE 
    ### social media ###
    -- anything from social
    WHEN LOWER(COALESCE(Media_Source,Partner,Campaign)) LIKE '%social%' THEN 'Social'
    ### direct ###
    -- fwb / referral
    WHEN COALESCE(Media_Source,Partner,Campaign) = 'af_app_invites' THEN 'FWB'
    WHEN (Media_Source = 'invalid_media_source_name' OR Media_Source IS NULL) AND LOWER(Campaign) LIKE '%refer%' THEN 'FWB'
    WHEN LOWER(COALESCE(Media_Source,Partner,Campaign)) LIKE '%refer%' THEN 'FWB'
    WHEN LOWER(COALESCE(Media_Source,Partner,Campaign)) LIKE '%moengage%' THEN 'FWB'
    -- apple search
    WHEN LOWER(COALESCE(Media_Source,Partner,Campaign)) LIKE '%apple search ads%' THEN 'AppleSearch'
    -- facebook / meta
    WHEN COALESCE(Media_Source,Partner,Campaign) LIKE '%Facebook Ads%' THEN 'Ads_Meta'
    WHEN COALESCE(Media_Source,Partner,Campaign) LIKE '%meta%' THEN 'Ads_Meta'
    WHEN COALESCE(Media_Source,Partner,Campaign) = 'facebook' THEN 'Ads_Meta'
    WHEN LOWER(COALESCE(Media_Source,Partner,Campaign)) = 'restricted' THEN 'Ads_Meta'
    -- google
    WHEN COALESCE(Media_Source,Partner,Campaign) LIKE '%googleadwords%' THEN 'Ads_Google'
      -- tiktok
    WHEN COALESCE(Media_Source,Partner,Campaign) like '%bytedanceglobal%' THEN 'TikTok_Paid'
    ### affiliates ###
    -- pokkt source
    WHEN LOWER(COALESCE(Media_Source,Partner,Campaign)) LIKE '%pokkt%' THEN 'Aff_Pokkt'
    WHEN COALESCE(Media_Source,Partner,Campaign) LIKE '%Tonik_CPA%' THEN 'Aff_Pokkt'
    WHEN COALESCE(Media_Source,Partner,Campaign) LIKE '%campaign_name%' THEN 'Aff_Pokkt'
    -- tyr ads
    WHEN LOWER(COALESCE(Media_Source,Partner,Campaign)) IN ('tyrads_int','tjzymob_int','tyrads','ta_tonik_aos_ph') THEN 'Aff_TyrAds'
    -- sales doubler
    WHEN LOWER(COALESCE(Media_Source,Partner,Campaign)) LIKE '%salesdoubler%' THEN 'Aff_SalesDoubler'
     
    -- imoney
    WHEN COALESCE(Media_Source,Partner,Campaign) LIKE '%imoney%' THEN 'Aff_iMoney'
    -- moneymax
    WHEN COALESCE(Media_Source,Partner,Campaign) LIKE '%moneymax%' THEN 'Aff_MoneyMax'
    -- jeff
    WHEN COALESCE(Media_Source,Partner,Campaign) LIKE '%jeff%' THEN 'Aff_Jeff'
    -- shareit
    WHEN COALESCE(Media_Source,Partner,Campaign) LIKE '%shareit%' THEN 'Aff_ShareIt'
     
    -- appnext
    WHEN COALESCE(Media_Source,Partner,Campaign) LIKE '%appnext%' THEN 'Aff_AppNext'
    -- mediadonuts / entravision
    WHEN LOWER(COALESCE(Media_Source,Partner,Campaign)) LIKE '%mediadonuts%' THEN 'Aff_Mediadonuts'
    WHEN LOWER(COALESCE(Media_Source,Partner,Campaign)) LIKE '%entravision%' THEN 'Aff_Mediadonuts'
    -- shopback
    WHEN LOWER(COALESCE(Media_Source,Partner,Campaign)) LIKE '%shopback_int%' THEN 'Aff_Shopback'
    -- deepsea
    WHEN LOWER(COALESCE(Media_Source,Partner,Campaign)) LIKE '%deepsea%' THEN 'Aff_DeepSea'
    ## organic ##
    WHEN COALESCE(Media_Source,Partner,Campaign) IN ('af_banner') THEN 'af_Banner'
    WHEN COALESCE(Media_Source,Partner,Campaign) LIKE '%sendgrid%' THEN 'Organic'
    WHEN COALESCE(Media_Source,Partner,Campaign) IN ('Website') THEN 'Website'
    
    ## owned media ##
    WHEN media_source IN ('PRFlexLoans_WebAndroidPressRelease') THEN 'Owned Media'
    
    WHEN media_source IN ('invalid_media_source_name') THEN 'Invalid Media Source'
    WHEN media_source IN ('None') THEN 'Broken OneLink'
ELSE "Organic"
END as Source,
    
digitalLoanAccountId,
flagApproval,
customerId,
Appsflyer_id
FROM `prj-prod-dataplatform.risk_credit_mis.loan_master_table` LA
WHERE 1=1 
AND DATE(LA.decision_date) >= '2023-07-01'
AND new_loan_Type IN ('Quick','Flex')
ORDER BY 1 
)
select customerId, Appsflyer_id, digitalLoanAccountId,Source  
, case when Source like 'Organic' then 'Organic' else 'InOrganic' end flag_organic_inorganic
from applications;"""

In [13]:
dfaquisition = client.query(sq).to_dataframe(progress_bar_type='tqdm')
dfaquisition.shape

Job ID 31f3a777-3312-4db4-af30-037f19cd60b5 successfully executed: 100%|██████████|
Downloading: 100%|██████████|


(455133, 5)

In [14]:
dfaquisition.columns

Index(['customerId', 'Appsflyer_id', 'digitalLoanAccountId', 'Source',
       'flag_organic_inorganic'],
      dtype='object')

# Query for the rest of the part

In [15]:
sq = """with 
devicemodel as 
(with base as
(select distinct lmt.digitalLoanAccountId,
  t4.GeneralInfo.brand     ,
  t4.Hardware.device__brand   ,
  t4.Hardware.device__manufacturer   ,
  t4.Hardware.device__model
from`risk_credit_mis.loan_master_table` lmt
LEFT JOIN
`prj-prod-dataplatform.dl_loans_db_raw.tdbk_digital_loan_application` t2
ON lmt.digitalLoanAccountId = t2.digitalLoanAccountId
LEFT JOIN
`prj-prod-dataplatform.dl_loans_db_raw.tdbk_credolab_track` t3
ON t2.credolabRefNumber = t3.refno
LEFT JOIN
`prj-prod-dataplatform.credolab_raw.android_credolab_datasets_struct_columns` t4
ON t3.refno = t4.deviceId
)
select digitalLoanAccountId, device__manufacturer devicemanufacturer, device__model devicemodel from base),
employmenttype as (select * from 
(select distinct digitalLoanAccountId, OccupationStatusDesc, row_number() over(partition by digitalLoanAccountId order by LastUpdateDate desc) rnk 
from prj-prod-dataplatform.risk_credit_cic_data.employment_data where OccupationStatusDesc is not null
)
where rnk = 1),
loanlocation as (select * from 
(select digitalLoanAccountId,
created_dt,
updated_dt,
latitude,
longitude,
row_number() over(partition by digitalLoanAccountid order by created_dt desc) rnk
 from prj-prod-dataplatform.dl_loans_db_raw.tdbk_customer_location
)
where rnk = 1),
base as 
(select * from
(select  distinct customerId,user_id,tsaOnboardingDateTime,credolab_ReferenceNumber,deviceType,
digitalLoanAccountId,case when loan_type in ('Flex-up','Reloan') then 'Repeat Applicant' 
when First_application=1 and customer_id_first_applicant is not null then 'New Applicant'
when accept_cust_id is null then  'Prev_Rejected_Reapply'
when rn=1 then 'New Applicant' 
--when Reject_application_count=1 then 'New Applicant' 
when accept_application_count>1 and accept_cust_id is not null then  'Repeat Applicant'

else 'Repeat Applicant' 
end as applicant_type,loan_type,loan_sub_type,sil_product,application_Submission_DateTime,
fraudScore,credolabScore,decision,applicationStatus,disbursementFlag,flg_mature_fpd10,flg_mature_fpd30,
flg_mature_fspd_30,flg_mature_fstpd_30,deffpd10,deffpd30,deffspd30,deffstpd30

from (with deliquency as
(select loanAccountNumber,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30  in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data),

accept_application_cnt as (select customerId,count(*) as accept_application_count
from risk_credit_mis.loan_master_table loan where loan.new_loan_type in ('Flex-up','SIL-Instore','SIL-Online(discontinued)','Quick','SIL ZERO','ACL Patner 711')
  and (flagApproval = 1 or flagRejection = 1 or loan.new_loan_type in ('Flex-up')) and coalesce(loan.termsAndConditionsSubmitDateTime,if (loan.new_loan_type ='Flex-up',loan.startApplyDateTime,loan.termsAndConditionsSubmitDateTime))  is not null and applicationStatus in ('ACTIVATED','COMPLETED','APPROVED',
'INPROGRESS','SIGNEDDOCCOMPLETED','ACCEPT') group by 1 order by customerId),


applciant_type_first as (select customerId customer_id_first_applicant,count(*) First_application from `prj-prod-dataplatform.risk_credit_mis.loan_master_table` 
where new_loan_type in ('Flex-up','SIL-Instore','SIL-Online(discontinued)','Quick','SIL ZERO','ACL Patner 711')
  and (flagApproval = 1 or flagRejection = 1 or new_loan_type in ('Flex-up')) and coalesce(termsAndConditionsSubmitDateTime,if (new_loan_type ='Flex-up',startApplyDateTime,termsAndConditionsSubmitDateTime))  is not null group by 1 having count(*)=1),

previous_loan_product as (select loan.customerId,
    case when loan.new_loan_type ='Flex-up' and reloan_flag=1 then 'Reloan'
    when loan.new_loan_type ='Flex-up' and reloan_flag=0 then 'Flex-up'
     WHEN loan.new_loan_type ='Flex' then 'Flex'
    when loan.new_loan_type ='Quick'  then 'Quick'
    when loan.new_loan_type ='Flex-up' and reloan_flag=0 then 'Flex-up'
    when loan.new_loan_type in ('SIL-Instore','SIL-Online(discontinued)','SIL ZERO') then 'SIL'
    when loan.new_loan_type in ('ACL Patner 711') then '711'
    end as prev_new_loan_type,loan.loanType prev_loan_type,prev_loan.previous_loan_acct,prev_loan.loanAccountNumber,loan.startApplyDateTime from `risk_credit_mis.loan_master_table` loan
join
(select * from (select customerId, digitalLoanAccountId,loanAccountNumber,new_loan_type,startApplyDateTime, lead(loanAccountNumber)
 OVER(PARTITION BY customerId order by startApplyDateTime desc) as previous_loan_acct from `risk_credit_mis.loan_master_table`
 where applicationStatus in ('COMPLETED','ACTIVATED')  order by customerId) where --previous_loan_acct is not null and 
 new_loan_type='Flex-up')prev_loan on loan.loanAccountNumber=prev_loan.previous_loan_acct)
  
  select
  loan.customerId,
  cust.user_id,
  tsa_onboarding_time as tsaOnboardingDateTime,
  credolabRefNumber as credolab_ReferenceNumber,
  deviceType,
  loan.digitalLoanAccountId,
  --case when applciant_type_first.disbursed_loan >1 then 'Repeat Applicant' else 'New Applicant' end as applicant_type,

    case when loan.new_loan_type ='Flex-up' and reloan_flag=1 then 'Reloan'
    when loan.new_loan_type ='Flex-up' and reloan_flag=0 then 'Flex-up'
    when loan.new_loan_type ='Quick'  then 'Quick'
    when loan.new_loan_type ='Flex-up' and reloan_flag=0 then 'Flex-up'
    when loan.new_loan_type in ('SIL-Instore','SIL-Online(discontinued)','SIL ZERO') then 'SIL'
    when loan.new_loan_type in ('ACL Patner 711') then '711'
    end as loan_type,
    case when loan.new_loan_type ='Flex-up' and reloan_flag=1 then coalesce('Reloan_'||prev_new_loan_type)
    when loan.new_loan_type ='Flex-up' and reloan_flag=0 then coalesce('Flex-up_'||prev_new_loan_type)
    else loan.new_loan_type end loan_sub_type,
    
    --new_loan_type as sub_loan_type,reloan_flag,
    --prev_new_loan_type, 
    case when loan.loantype='BNPL' and store_type =1 then 'Appliance'
    when loan.loantype='BNPL' and store_type =2 then 'Mobile' 
    when loan.loantype='BNPL' and store_type =3 then 'Mall' 
    when loan.loantype='BNPL' and store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as sil_product,
    coalesce(loan.termsAndConditionsSubmitDateTime,if (loan.new_loan_type ='Flex-up',loan.startApplyDateTime,loan.termsAndConditionsSubmitDateTime)) application_Submission_DateTime,
    fraudScore,credolabScore,
      case
      when flagApproval = 1 then 'approved'
      when flagRejection = 1 then 'rejected'
      else 'auto-approved without model'
    end as decision,
    applicationStatus,
    flagDisbursement as disbursementFlag,
    IF(Date_diff(current_date(),date(disbursementDateTime), DAY) >= 40, 1 ,0) as flg_mature_fpd10,
    IF(Date_diff(current_date(),date(disbursementDateTime), DAY) >= 60, 1 ,0) as flg_mature_fpd30,
    IF(Date_diff(current_date(),date(disbursementDateTime), DAY) >= 90, 1 ,0) as flg_mature_fspd_30,
    IF(Date_diff(current_date(),date(disbursementDateTime), DAY) >= 120, 1 ,0) as flg_mature_fstpd_30,
    deliquency.deffpd10,deliquency.deffpd30,deliquency.deffspd30,deliquency.deffstpd30,row_number() over (partition by loan.customerId order by coalesce(termsAndConditionsSubmitDateTime,if (new_loan_type ='Flex-up',loan.startApplyDateTime,termsAndConditionsSubmitDateTime))) rn,accept_application_count,accept_application_cnt.customerId accept_cust_id,First_application,customer_id_first_applicant

   
  from prj-prod-dataplatform.risk_credit_mis.loan_master_table loan
  left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name) sil_category on purpleKey=sil_category.mer_refferal_code
  left join deliquency on deliquency.loanAccountNumber=loan.loanAccountNumber
  left join previous_loan_product on previous_loan_product.loanAccountNumber=loan.loanAccountNumber
  left join applciant_type_first on applciant_type_first.customer_id_first_applicant=loan.customerId
  left join dl_customers_db_raw.tdbk_customer_mtb cust on cust.cust_id=cast(loan.customerId as string)
  left join accept_application_cnt on accept_application_cnt.customerId=loan.customerId
  where loan.new_loan_type in ('Flex-up','SIL-Instore','SIL-Online(discontinued)','Quick','SIL ZERO','ACL Patner 711')
  and (flagApproval = 1 or flagRejection = 1 or loan.new_loan_type in ('Flex-up')) and coalesce(loan.termsAndConditionsSubmitDateTime,if (loan.new_loan_type ='Flex-up',loan.startApplyDateTime,loan.termsAndConditionsSubmitDateTime))  is not null) final where loan_sub_type is not null)
), 
delqtab as 
(select loanAccountNumber,  
case when obs_min_inst_def30 >=2 then 1 else 0 end obsFSPD30,
case when min_inst_def30 in (1,2) then 1 else 0 end fspd30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data),
Excl_evertransacted as 
(select distinct cast(customer_id as numeric) customerId from prj-prod-dataplatform.risk_mart.customer_transactions where status like 'Success'),
Excl_qualified_FSPD30 as
(select loanAccountNumber, obs_min_inst_def30 from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data where obs_min_inst_def30 >= 2),
Excl_blacklisted_clean as 
(select customerid, matching_attribute, id, refresh_date from  prj-prod-dataplatform.CRM_RISK.fraud_blacklist_clean),
educate as 
    (select distinct edu.digitalLoanAccountId, edu.education_id, edu1.description
    from `prj-prod-dataplatform.dl_loans_db_raw.tdbk_loan_purpose` edu
    inner join (select id, description from dl_loans_db_raw.tdbk_loan_lov_mtb where module = 'Education') edu1 on edu.education_id = edu1.id
    ),
    educate2 as 
    (select *, row_number() over(partition by digitalLoanAccountId order by education_id desc) rnk from educate),
    educate3 as 
    (select * from educate2 where rnk = 1),
    education as 
    (
    select 
    lmt.customerId,
    educate3.digitalLoanAccountId,
    educate3.education_id,
    educate3.description Education_Type, 
    educate3.rnk educationrnk
    from educate3 
    inner join prj-prod-dataplatform.risk_credit_mis.loan_master_table lmt on lmt.digitalLoanAccountId = educate3.digitalLoanAccountId
    )
    ,
    digitalloanapplication as (select 
    digitalLoanAccountId, 
    monthlyInterestRate , 
    interest_rate_offer ,
    monthlyIncomeCheck, 
    from prj-prod-dataplatform.dl_loans_db_raw.tdbk_digital_loan_application ), 
    loantypeselection as 
    (SELECT 
    digitalLoanAccountId 
    FROM `prj-prod-dataplatform.risk_credit_mis.loan_master_table` 
    where date(disbursementDateTime) >= '2023-07-01'
    and flagDisbursement = 1
    and kycStatus = 'SKYC'
    -- and customerId in (select customerId from Excl_evertransacted) ---Excluded customers who never transacted
    -- and loanAccountNumber in (select loanAccountNumber from Excl_qualified_FSPD30) --- Exclude customer who has not yet matured FSPD30
    -- and customerId not in (select distinct customerid from Excl_blacklisted_clean) --- Exclude customer who are blacklisted
    -- and customerId not in (select distinct customerId from `risk_credit_mis.loan_master_table` where loanPaidStatus in ('Written Off', 'Sold')) --- Exclude written off
    -- and customerId not in (Select PID from dap_ds_poweruser_playground.Defraudedandhacked_mu where PID is not null)
    ),
    Appreinstall as 
    (select cast(customer_id as numeric) customerId, count(customer_id)  from moengage_raw.events_hourly where event_code = 'REINSTALL'
        group by 1
        having count(customer_id) > 1)
    ,
    loan
    as
    (select 
    distinct
    lmt.digitalLoanAccountId ,
    lmt.customerId ,
    lmt.loanAccountNumber ,
    lmt.tsa_onboarding_time,
    lmt.startApplyDateTime,
    lmt.decision_date,
    lmt.termsAndConditionsSubmitDateTime,
    lmt.disbursementDateTime, 
    lmt.new_loan_type,
    lmt.gender,
    lmt.age,
    education.Education_Type,
    education.education_id,
    et.OccupationStatusDesc Employment_type,
    lmt.custLoanJourneyCity,
    lmt.custLoanJourneyProvince,
    lmt.custLoanJourneyZipCode,
    ll.latitude loanapplication_latitude,
    ll.longitude loanapplication_longitude,
    lmt.province,
    lmt.city,
    lmt.postalCode,
    lmt.geolocation onboardinggeolocation,
    lmt.incomeperSalaryScale,
    lmt.monthlyIncome,
    lmt.maritalStatus,
    lmt.purposeDescription,
    ascv.mer_category,
    lmt.isfatca,
    lmt.docType,
    lmt.loanRequestAmount,
    lmt.loanRequestTenure,
    dlp.monthlyInterestRate,
    dlp.interest_rate_offer,
    dlp.monthlyIncomeCheck,
    lmt.kycStatus,
	case when lower(coalesce(lmt.osVersion, lmt.osversion_v2)) like '%ios%' then 'ios'
	     when lower(coalesce(lmt.osVersion, lmt.osversion_v2)) like '%andro%' then 'android'
		 else 'other' end Device_os_type,
    delqtab.obsFSPD30,
    delqtab.FSPD30,
	  case when lower(coalesce(lmt.osVersion, lmt.osversion_v2)) like '%ios%' then coalesce(cast(SUBSTRING(coalesce(lmt.osVersion, lmt.osversion_v2), 4, 2)  as numeric),0)
	     when lower(coalesce(lmt.osVersion, lmt.osversion_v2)) like '%andro%' then coalesce(cast(SUBSTRING(coalesce(lmt.osVersion, lmt.osversion_v2), 8, 2)  as numeric),0)
		 end Device_OSVersion,
   lmt.deviceType,
    b.Network Network_provider,
    lmt.brand,
    dm.devicemanufacturer,
    dm.devicemodel,
	case when eet.customerId is null then 1 else 0 end customer_never_transacted_flag,
	case when eqf.loanAccountNumber is not null then 1 else 0 end fspd30_mature_flag,
	case when ebc.customerId is not null then 1 else 0 end blacklisted_customer_flag,
	case when lmt.customerId in (select distinct customerId from `risk_credit_mis.loan_master_table` where loanPaidStatus in ('Written Off', 'Sold')) then 1 else 0 end writtenoff_flag,
	case when lmt.customerId in (Select PID from dap_ds_poweruser_playground.Defraudedandhacked_mu where PID is not null) then 1 else 0 end hackedordefrauded_flag,
  case when e.vas_flag = 'true' then 1 else 0 end as VasFlag,
  case when Appreinstall.customerId is not null then 1 else 0 end EverReinstallFlag,
  lmt.credolabScore
    from base 
    left join   `risk_credit_mis.loan_master_table` lmt on lmt.digitalLoanAccountId = base.digitalLoanAccountId
    inner join loantypeselection lts on lts.digitalLoanAccountId = lmt.digitalLoanAccountId 
    inner join delqtab on delqtab.loanAccountNumber = lmt.loanAccountNumber
    Left join education on education.digitalLoanAccountId = lmt.digitalLoanAccountId
    left join digitalloanapplication dlp on dlp.digitalLoanAccountId = lmt.digitalLoanAccountId
    LEFT JOIN (select *, row_number() over(partition by Number_Prefix order by Network) rnk 
  from `prj-prod-dataplatform.manual_source_extracts.mobile_carrier_mapping`) b ON LEFT(RIGHT(lmt.mobileNo,LENGTH(lmt.mobileNo)-2),3) = CAST(b.number_prefix AS STRING)
	 left join Excl_evertransacted eet on eet.customerId = lmt.customerId
	 left join Excl_qualified_FSPD30 eqf on eqf.loanAccountNumber = lmt.loanAccountNumber
	 left join Excl_blacklisted_clean ebc on ebc.customerId = lmt.customerId
   left join prj-prod-dataplatform.dl_loans_db_raw.tdbk_digital_loan_application e
          on e.loanAccountNumber = lmt.loanAccountNumber
   left join Appreinstall on Appreinstall.customerId = lmt.customerId
   left join prj-prod-dataplatform.worktable_datachampions.AREA_SPLIT_CURRENTV2 ascv on ascv.purpleKey = lmt.purpleKey
   left join loanlocation ll on ll.digitalLoanAccountId = lmt.digitalLoanAccountId
   left join employmenttype et on et.digitalLoanAccountId = lmt.digitalLoanAccountId
   left join devicemodel dm on dm.digitalLoanAccountId = lmt.digitalLoanAccountId
    where b.rnk = 1
    )
    select * from loan ;"""

In [16]:
df = client.query(sq).to_dataframe(progress_bar_type='tqdm')

Job ID 67efae2c-57df-45e8-aa92-ddf2331b1ec9 successfully executed: 100%|██████████|
Downloading: 100%|██████████|


In [17]:
df.shape 

(134946, 53)

# Events Before Apply

In [18]:
sq = """with eventbase as 
(select Customer_User_ID,Event_Time, Event_Name  from prj-prod-dataplatform.appsflyer_raw.in_app_events_report where Customer_User_ID is not null 
union all
select Customer_User_ID,Event_Time, Event_Name from prj-prod-dataplatform.appsflyer_raw.organic_in_app_events_report where Customer_User_ID is not null 
),
eventsbeforeapply as 
(select a.customerId, a.digitalLoanAccountId , a.startApplyDateTime 
, eb.Customer_User_ID, eb.Event_Time, eb.Event_Name
from `risk_credit_mis.loan_master_table` a
left join eventbase eb on eb.Customer_User_ID  = cast(a.customerId as string) and date(a.startApplyDateTime) > date(eb.Event_Time)
),
eventsbeforeapplybase as
(select digitalLoanAccountId, startApplyDateTime, count(distinct Event_Name) cntevents 
from eventsbeforeapply
group by 1,2
)
select * from eventsbeforeapplybase;"""

eventbfrapplydf = client.query(sq).to_dataframe(progress_bar_type='tqdm')
eventbfrapplydf

Job ID 0510792b-f264-42b4-b15f-a6cd7d407d09 successfully executed: 100%|██████████|
Downloading: 100%|██████████|


,digitalLoanAccountId,startApplyDateTime,cntevents
0,67113400-31c1-4e54-9283-1e05a8daca21,2024-03-14 12:29:37,176
1,ef9b2de9-0fe9-4537-9603-70db6156b882,2024-02-29 08:37:48,167
2,1c4c1a2b-92b9-4a7e-ae6f-5334665d9343,2023-08-24 17:27:04,68
3,f56d9d64-9c57-434e-b4e0-388b912f83ee,2024-02-05 16:31:54,96
4,93cf5269-bd46-4141-bbef-ba83ec4feefe,2023-07-30 15:41:01,71
...,...,...,...
1819181,1bfe3d3b-beb5-4396-b74d-b7e9243a3e64,2023-10-01 14:32:08,0
1819182,da167bba-5a13-4bc7-b84f-892cef7870e9,2024-07-29 18:40:59,0
1819183,84c5f5f1-0d3a-4ccc-9dd1-0590d992a401,2024-02-14 09:23:16,0
1819184,b74da09c-4c25-4e1c-a7d3-721b8eb8b46f,2024-08-04 16:02:02,0


# Saving the dataframes to csv files

In [19]:
dfhacked1.to_csv(r"C:\Users\DwaipayanChakroborti\OneDrive - Tonik Financial Pte Ltd\MyStuff\Biswa\Demo&Telco Scorecard\Data\hackedconfirmedbyoperationsteam.csv", index = False)
dfaquisition.to_csv(r"C:\Users\DwaipayanChakroborti\OneDrive - Tonik Financial Pte Ltd\MyStuff\Biswa\Demo&Telco Scorecard\Data\aquisitiondatareceivedfromIvanah.csv", index = False)
df.to_csv(r"C:\Users\DwaipayanChakroborti\OneDrive - Tonik Financial Pte Ltd\MyStuff\Biswa\Demo&Telco Scorecard\Data\otherpart.csv", index = False)
eventbfrapplydf.to_csv(r"C:\Users\DwaipayanChakroborti\OneDrive - Tonik Financial Pte Ltd\MyStuff\Biswa\Demo&Telco Scorecard\Data\eventsbeforeapplydate.csv", index = False)

In [20]:
finaldf = df.merge(dfaquisition, on='digitalLoanAccountId', how = 'left')
finaldf = finaldf.merge(eventbfrapplydf[['digitalLoanAccountId', 'cntevents']], on='digitalLoanAccountId', how = 'left')
finaldf.shape

(134946, 58)

In [21]:
finaldf.sample(5)

,digitalLoanAccountId,customerId_x,loanAccountNumber,tsa_onboarding_time,startApplyDateTime,decision_date,termsAndConditionsSubmitDateTime,disbursementDateTime,new_loan_type,gender,age,Education_Type,education_id,Employment_type,custLoanJourneyCity,custLoanJourneyProvince,custLoanJourneyZipCode,loanapplication_latitude,loanapplication_longitude,province,city,postalCode,onboardinggeolocation,incomeperSalaryScale,monthlyIncome,maritalStatus,purposeDescription,mer_category,isfatca,docType,loanRequestAmount,loanRequestTenure,monthlyInterestRate,interest_rate_offer,monthlyIncomeCheck,kycStatus,Device_os_type,obsFSPD30,FSPD30,Device_OSVersion,deviceType,Network_provider,brand,devicemanufacturer,devicemodel,customer_never_transacted_flag,fspd30_mature_flag,blacklisted_customer_flag,writtenoff_flag,hackedordefrauded_flag,VasFlag,EverReinstallFlag,credolabScore,customerId_y,Appsflyer_id,Source,flag_organic_inorganic,cntevents
96784,72f3e190-d710-4e2e-877d-4660d93bff80,2713578,60827135780012,2024-08-02 11:29:13,2024-08-02 11:40:12,2024-08-02 11:43:06,2024-08-02 11:42:55,2024-08-02 11:51:23,SIL-Instore,F,57,College Undergraduate,13,None,CITY OF PARANAQUE,METRO MANILA,1700,14.505636,121.040790,METRO MANILA,CITY OF PARANAQUE,1700,14.4918521 | 121.0437821,40000,40000,Widow / Widower,"Sala set, Couch, Sofa set,Air conditioners",Appliances Retailer,0,Philippines - Driving License (2022),21871.0,12,4.29,None,None,SKYC,android,0,0,14.000000000,Android,Smart,samsung,None,SM-A155F,0,0,0,0,0,0,0,410.0,<NA>,NaN,NaN,NaN,0
49930,98e7a05b-64fa-4c11-aeed-1a95d1fcbdb7,2334695,60823346950016,2023-12-15 12:38:51,2023-12-15 12:49:53,2023-12-15 13:00:02,2023-12-15 12:59:57,2023-12-15 13:08:38,SIL-Instore,M,54,College Undergraduate,13,None,TRECE MARTIRES CITY,CAVITE,4109,14.280900,120.868884,Cavite,TRECE MARTIRES CITY,4109,14.2809304 | 120.8687552,32747.00,21000,Married,"Oven, Cooking Range, Griller,Bed frames or Cribs",Appliances Retailer,0,Philippines - Driving License,5015.0,3,4.50,None,true,SKYC,android,1,0,13.000000000,Android,Smart,OPPO,None,CPH2325,0,1,0,0,0,0,0,469,<NA>,NaN,NaN,NaN,0
30616,3524f1cf-3bcb-4097-a021-50ce22a8dfb8,2309698,60823096980014,2023-11-20 12:54:33,2023-11-20 12:58:31,2023-11-20 13:04:57,2023-11-20 13:04:52,2023-11-20 13:19:36,SIL-Instore,F,38,High School Graduate,11,None,CITY OF VALENZUELA,METRO MANILA,1448,14.709220,120.993550,METRO MANILA,CITY OF VALENZUELA,1448,14.7065352 | 121.0072266,19171.00,28000,Live-in Partner,"Stereo set, Soundbar, Audio equipment",Appliances Retailer,0,Philippines - Id Card (2020),5998.0,6,4.50,None,true,SKYC,android,1,0,11.000000000,Android,Globe/TM,vivo,None,vivo 1906,0,1,0,0,0,1,0,474,<NA>,NaN,NaN,NaN,0
88064,cc95c2c5-f40e-4f78-9ef4-95ca4b2f6ec8,2503522,60825035220019,2024-05-07 19:58:27,2024-05-07 20:01:07,2024-05-07 20:07:51,2024-05-07 20:07:45,2024-05-08 13:39:44,SIL-Instore,F,39,College Graduate,14,None,CITY OF MAKATI,METRO MANILA,1200,14.547081,120.998171,Batangas,TAAL,4208,14.5502383 | 121.009615,30535.00,32000,Married,Air conditioners,Appliances Retailer,0,Philippines - UMID Card (2016),7999.0,6,6.52,None,true,SKYC,android,1,0,11.000000000,Android,DITO,realme,None,RMX2061,0,1,0,0,0,1,0,423,<NA>,NaN,NaN,NaN,0
94280,351ecd4f-91f0-4050-89c4-cdd4e927ec7d,2489027,60824890270017,2024-04-29 18:53:19,2024-04-29 18:56:02,2024-04-29 18:58:40,2024-04-29 18:58:35,2024-04-29 19:11:10,SIL-Instore,M,42,College Undergraduate,13,Not Employed,CITY OF TAGUIG,METRO MANILA,1630,14.507302,121.053327,METRO MANILA,CITY OF TAGUIG,1630,14.5073025 | 121.0533271,34460.00,32000,Married,Air conditioners,Appliances Retailer,0,Philippines - Driving License (2022),16399.0,8,5.04,None,true,SKYC,android,1,0,14.000000000,Android,TNT,vivo,None,V2237,0,1,0,0,0,1,0,467,<NA>,NaN,NaN,NaN,0


In [24]:
finaldf.columns

Index(['digitalLoanAccountId', 'customerId', 'loanAccountNumber',
       'tsa_onboarding_time', 'startApplyDateTime', 'decision_date',
       'termsAndConditionsSubmitDateTime', 'disbursementDateTime',
       'new_loan_type', 'gender', 'age', 'Education_Type', 'education_id',
       'Employment_type', 'custLoanJourneyCity', 'custLoanJourneyProvince',
       'custLoanJourneyZipCode', 'loanapplication_latitude',
       'loanapplication_longitude', 'province', 'city', 'postalCode',
       'onboardinggeolocation', 'incomeperSalaryScale', 'monthlyIncome',
       'maritalStatus', 'purposeDescription', 'mer_category', 'isfatca',
       'docType', 'loanRequestAmount', 'loanRequestTenure',
       'monthlyInterestRate', 'interest_rate_offer', 'monthlyIncomeCheck',
       'kycStatus', 'Device_os_type', 'obsFSPD30', 'FSPD30',
       'Device_OSVersion', 'deviceType', 'Network_provider', 'brand',
       'devicemanufacturer', 'devicemodel', 'customer_never_transacted_flag',
       'fspd30_mature_flag

In [23]:
finaldf.rename(columns={'customerId_x':'customerId'}, inplace = True)


In [25]:
finaldf = finaldf[['digitalLoanAccountId', 'customerId', 'loanAccountNumber',
       'tsa_onboarding_time', 'startApplyDateTime', 'decision_date',
       'termsAndConditionsSubmitDateTime', 'disbursementDateTime',
       'new_loan_type', 'gender', 'age', 'Education_Type', 'education_id',
       'Employment_type', 'custLoanJourneyCity', 'custLoanJourneyProvince',
       'custLoanJourneyZipCode', 'loanapplication_latitude',
       'loanapplication_longitude', 'province', 'city', 'postalCode',
       'onboardinggeolocation', 'incomeperSalaryScale', 'monthlyIncome',
       'maritalStatus', 'purposeDescription', 'mer_category', 'isfatca',
       'docType', 'loanRequestAmount', 'loanRequestTenure',
       'monthlyInterestRate', 'interest_rate_offer', 'monthlyIncomeCheck',
       'kycStatus', 'Device_os_type', 'obsFSPD30', 'FSPD30',
       'Device_OSVersion', 'deviceType', 'Network_provider', 'brand',
       'devicemanufacturer', 'devicemodel', 'customer_never_transacted_flag',
       'fspd30_mature_flag', 'blacklisted_customer_flag', 'writtenoff_flag',
       'hackedordefrauded_flag', 'VasFlag', 'EverReinstallFlag',
       'credolabScore', 'Appsflyer_id', 'Source',
       'flag_organic_inorganic', 'cntevents']].copy()

In [26]:
finaldf.sample(5)

,digitalLoanAccountId,customerId,loanAccountNumber,tsa_onboarding_time,startApplyDateTime,decision_date,termsAndConditionsSubmitDateTime,disbursementDateTime,new_loan_type,gender,age,Education_Type,education_id,Employment_type,custLoanJourneyCity,custLoanJourneyProvince,custLoanJourneyZipCode,loanapplication_latitude,loanapplication_longitude,province,city,postalCode,onboardinggeolocation,incomeperSalaryScale,monthlyIncome,maritalStatus,purposeDescription,mer_category,isfatca,docType,loanRequestAmount,loanRequestTenure,monthlyInterestRate,interest_rate_offer,monthlyIncomeCheck,kycStatus,Device_os_type,obsFSPD30,FSPD30,Device_OSVersion,deviceType,Network_provider,brand,devicemanufacturer,devicemodel,customer_never_transacted_flag,fspd30_mature_flag,blacklisted_customer_flag,writtenoff_flag,hackedordefrauded_flag,VasFlag,EverReinstallFlag,credolabScore,Appsflyer_id,Source,flag_organic_inorganic,cntevents
93164,942c3922-426a-4eec-bb06-05d40df7c896,2354484,60823544840017,2023-12-31 09:54:54,2023-12-31 09:56:41,2023-12-31 10:03:26,2023-12-31 10:03:22,2023-12-31 10:19:10,SIL-Instore,M,28,High School Graduate,11,None,SANTA MARIA,BULACAN,3022,14.847536,120.980696,Bulacan,SANTA MARIA,3022,null | null,17754.00,20000,Single,"Washing machine, dryer or both",Appliances Retailer,0,Philippines - UMID Card (2016),7905.0,3,4.500000,None,true,SKYC,android,1,0,13.000000000,Android,Smart,Redmi,None,23021RAAEG,0,1,0,0,0,0,0,459,NaN,NaN,NaN,0
98229,c92a11d7-3c37-4a1a-9dcc-1882c264fde1,2354730,60823547300012,2023-12-31 12:56:41,2023-12-31 12:58:59,2023-12-31 13:02:47,2023-12-31 13:02:42,2023-12-31 13:08:31,SIL-Instore,M,38,Technical/Vocational Graduate,12,None,TANZA,CAVITE,4108,14.393249,120.852394,Cavite,TANZA,4108,14.3207931 | 120.8732063,26585.00,30000,Married,Refrigerators,Appliances Retailer,0,Philippines - UMID Card (2016),13799.0,6,4.500000,None,true,SKYC,android,1,0,13.000000000,Android,TNT,OPPO,None,2201117TG,0,1,0,0,0,1,0,487,NaN,NaN,NaN,0
41152,8866064e-4c26-4ea3-bd8a-2ca0f1e8ae0e,2237101,60822371010015,2023-09-17 14:21:12,2023-09-17 14:25:01,2023-09-17 14:29:16,2023-09-17 14:29:13,2023-09-17 14:37:55,SIL-Instore,F,30,None,None,None,TANZA,CAVITE,4108,14.393208,120.852422,Cavite,TANZA,4108,14.383588 | 120.8312039,22226.00,12000,Live-in Partner,Televisions,Appliances Retailer,0,Philippines - UMID Card (2011),7699.0,6,4.500000,None,true,SKYC,android,1,0,13.000000000,Android,Globe/TM,samsung,None,Aqua S10 Pro,0,1,0,0,0,0,0,453.0,NaN,NaN,NaN,0
101538,7535b441-ac8b-49db-a44e-8062a8b2c149,2771357,60827713570013,2024-08-19 18:32:28,2024-08-19 19:01:21,2024-08-19 19:11:02,2024-08-19 19:08:47,2024-08-19 19:56:10,Quick,M,53,College Graduate,14,Other,SOCORRO,ORIENTAL MINDORO,5207,14.605441,121.001382,Mindoro Oriental,SOCORRO,5207,null | null,53255.00,28000,Single,Medical or Other Emergency/Health Care,None,0,Philippines - UMID Card (2010),5000.0,6,7.916667,None,true,SKYC,android,0,0,14.000000000,Android,Sun,OPPO,None,CPH2625,0,0,0,0,0,0,0,432.0,None,Organic,Organic,0
14461,2e4ac0ac-b9e4-4385-aeaf-44018fb2431c,1106194,60811061940014,2021-10-24 17:30:12,2023-07-31 19:40:38,2023-08-01 10:07:08,2023-07-31 19:47:01,2023-08-01 10:11:31,Quick,M,39,None,None,None,POLOMOLOK,SOUTH COTABATO,9504,6.265969,125.013456,South Cotabato,POLOMOLOK,9504,6.2683968 | 125.01377,27183.00,27573,Single,Medical or Other Emergency/Health Care,None,0,Philippines - ePassport (2016),15000.0,12,7.000000,None,true,SKYC,android,1,0,12.000000000,Android,TNT,vivo,None,vivo 2019,0,1,0,0,0,0,0,523.0,1687787396918-6684931023294074689,Organic,Organic,47


In [27]:
finaldf.rename(columns= {'Source':'AquisitionChannel'}, inplace = True)

In [28]:
finaldf.to_csv(r"C:\Users\DwaipayanChakroborti\OneDrive - Tonik Financial Pte Ltd\MyStuff\Biswa\Demo&Telco Scorecard\Data\demographic_data_20241016.csv", index = False)

In [29]:
finaldf.dtypes

digitalLoanAccountId                        object
customerId                                   Int64
loanAccountNumber                           object
tsa_onboarding_time                 datetime64[us]
startApplyDateTime                  datetime64[us]
decision_date                       datetime64[us]
termsAndConditionsSubmitDateTime    datetime64[us]
disbursementDateTime                datetime64[us]
new_loan_type                               object
gender                                      object
age                                          Int64
Education_Type                              object
education_id                                object
Employment_type                             object
custLoanJourneyCity                         object
custLoanJourneyProvince                     object
custLoanJourneyZipCode                      object
loanapplication_latitude                   float64
loanapplication_longitude                  float64
province                       

In [30]:
finaldf.mer_category.unique()                               

array([None, 'Appliances Retailer', 'Mobile Retailer',
       'Home Appliances Retailer', 'Home and Furnitures Retailer'],
      dtype=object)

In [31]:
finaldf.to_pickle(r"C:\Users\DwaipayanChakroborti\OneDrive - Tonik Financial Pte Ltd\MyStuff\Biswa\Demo&Telco Scorecard\Data\demographic_data_20241016.pkl")